In [1]:
# Imports
import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp
from IPython.display import display
import json
from dataclasses import asdict

In [2]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=21000)
# Coherent source
cc = coordinator.register_component(module="color_center.py", runtime="python")
# Run the coordinator process
coordinator.run()

{'b0': {'value': None, 'type': 'number'}, 'temp': {'value': None, 'type': 'number'}}


In [3]:
cc.set_param("b0", 1)
cc.set_param("temp", 2)
cc.send_params()

In [4]:
cc_state = cc.state_init()[0]

In [5]:
print(cc_state)

### Classical signal construction
We import the Classical signal defined by the color center component in order to define the signal

In [6]:
from color_center import ClassicalSignal

sig = ClassicalSignal(
    b_amplitude=1,
    frequency=2.87 * 1e9, # Hz
    phase=0,
    width=1, #ns
)

In [7]:
response, operators = cc.channel_query(
    cc_state,
    {
        "state": cc_state.state_props[0].uuid
    },
    signals = [asdict(sig)],
    time = 0.01
)

{'msg_type': 'channel_query_response', 'kraus_operators': [[[[0.0, 0.0], [0.0, 0.0]], [[1.0, 0.0], [0.0, 0.0]]], [[[0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [1.0, 0.0]]]], 'kraus_state_indices': ['ad1b1834-b0bb-417a-95e7-74fd5050b55f'], 'error': 0, 'operation_time': 1, 'retrigger': False, 'sent_from': 53239}


In [8]:
cc_state.apply_kraus_operators(
    operators,
    cc_state.get_all_props(response["kraus_state_indices"]))

In [ ]:
cc_state.